# The reduced decision making model by Wong-Wang
The equations and parameters are described in the body, appendix and supplementary materials of the following paper:

<i>Wong, Kong-Fatt and Xiao-Jing Wang.</i> <b>"A recurrent network mechanism  of time integration in perceptual decisions"</b>. Journal of Neuroscience 26, no. 4 (2006)

In [25]:
import numpy as np
import random

#Single-cell input-output relation [Appendix & Supplementary material D]
def input_output_relation(state,a=270,b=108,d=0.154):
    return (a*state - b)/(1-np.exp(-d*(a*state - b)))
#H(x) = (a * x - b) / (1 - exp(-d*(a * x - b)))

#Noisy sources from each ionic channel 
def noisy_channel(noises,tau_AMPA = 2, s_noise=0.05):
    #a two-dimensional array of noisy displacements for the current run
    eta =  np.random.rand(2) * np.sqrt(tau_AMPA * (s_noise ** 2))
    #Inoise1' = -Inoise1 + eta1 * sqrt(tau_AMPA * s_noise^2)
    #Inoise2' = -Inoise2 + eta2 * sqrt(tau_AMPA * s_noise^2)
    return -noises + eta #[ionic_noises]

#Summing contributions to the hidden state (x) of each population
#x1 = JN11 * S1 - JN12 * S2 + I0 + I1 + Inoise1
#x2 = JN22 * S2 - JN21 * S1 + I0 + I2 + Inoise2

def build_hidden_states(couplings,current_states,inputs,ionic_noises,external_sources = 0.3255):
    #returns a 2x1 array, each element being the hidden state activity (firing rate) of one excitatory population
    return couplings @ current_states + inputs + ionic_noises + external_sources

#yet to port/solve

#these two will become the function below
#S1' = -S1/tau_S + (1 - S1) * g * H(x1)/1000
#S2' = -S2/tau_S + (1 - S2) * g * H(x2)/1000

def update_states(hidden_states,current_states,g=0.641,tau_S = 100):
    return -1*(current_states/tau_S) + (np.ones(2) - current_states) * g * input_output_relation(hidden_states)/1000

In [23]:
#Parameters
mu_0 = 30 #input strength; motion strength
c = 0 #input coherence

#Parameters to the input-output function
a = 270 #[(VnC)^{-1}]
b = 108 #[Hz]
d = 0.154 #[s]

#Kinetic parameters
g = 0.641 #dimensionless, called gamma in the paper
tau_S = 100 #[ms]
tau_AMPA = 2 #[ms]

#Synaptic coupling (population autapses {11,22} and between populations {21,12})
#JN11 = 0.2609
#JN22 = 0.2609
self_coupling = 0.2609
#JN21 = 0.0497
#JN12 = 0.0497
outer_coupling = 0.0497
couplings = [self_coupling,-outer_coupling]
#a 2x2 matrix whose main diagonal is comprised of self_coupling and whose secondary diagonal is -outer_coupling
couplings_matrix = [couplings,couplings[::-1]]

#Inputs from other sources (inhibitory and neutral populations: JA_ext; external: I0)
JA_ext = 5.2*np.exp(-4)
I0 = 0.3255

#Simulation noise
s_noise=0

#Visual motion stimulus to to the i-th {1,2} population 
I1 = JA_ext * mu_0 * (1 + c/100)
I2 = JA_ext * mu_0 * (1 - c/100)
inputs = np.array([I1,I2])

#Initial activity (firing rate) of each population
S1 = 0
S2 = 0
population_states = np.array([S1,S2])
current_noise = np.zeros(2)

#Simulation
sim_steps = 20_000
dt = 0.1
timestamps = []
time = 0

In [28]:
for i in range(sim_steps):
    time+=dt
    timestamps.append(time)
    current_noise = noisy_channel(current_noise)
    hidden_states = build_hidden_states(couplings=couplings_matrix,current_states=population_states,inputs=inputs,
                                    ionic_noises=current_noise,external_sources = I0)
    #print(hidden_states)
    difference = update_states(hidden_states,current_states=population_states)
    population_states += dt*difference
    print(population_states)

[0.98125366 0.9812469 ]
[0.98125357 0.98124631]
[0.98125429 0.98124636]
[0.98125413 0.98124628]
[0.98125451 0.98124556]
[0.98125483 0.98124472]
[0.98125504 0.98124392]
[0.98125511 0.98124413]
[0.98125513 0.98124366]
[0.98125526 0.98124416]
[0.98125503 0.98124545]
[0.98125557 0.98124559]
[0.98125604 0.98124637]
[0.9812565  0.98124715]
[0.98125779 0.98124727]
[0.98125819 0.98124729]
[0.98125928 0.98124848]
[0.98126062 0.98124962]
[0.98126245 0.98125107]
[0.98126451 0.98125124]
[0.98126614 0.98125237]
[0.98126667 0.98125335]
[0.98126748 0.98125444]
[0.9812689  0.98125622]
[0.98127014 0.98125791]
[0.98127017 0.98125967]
[0.98126933 0.98126065]
[0.98126828 0.9812625 ]
[0.98126735 0.98126308]
[0.98126625 0.98126313]
[0.98126444 0.98126423]
[0.98126379 0.98126443]
[0.98126312 0.98126534]
[0.98126276 0.98126523]
[0.98126354 0.98126484]
[0.98126491 0.9812642 ]
[0.9812665  0.98126421]
[0.98126765 0.98126486]
[0.98126816 0.98126613]
[0.98126837 0.98126683]
[0.98126857 0.9812682 ]
[0.98126878 0.98

[0.98123964 0.98123831]
[0.9812372 0.9812376]
[0.98123625 0.98123673]
[0.98123647 0.98123569]
[0.98123642 0.98123412]
[0.98123548 0.98123232]
[0.98123506 0.98123142]
[0.98123562 0.98123105]
[0.98123579 0.98123184]
[0.98123718 0.98123153]
[0.98123763 0.9812321 ]
[0.9812379  0.98123166]
[0.98123862 0.98123183]
[0.98124044 0.98123243]
[0.98124265 0.98123379]
[0.98124559 0.98123427]
[0.98124881 0.98123386]
[0.98125129 0.98123421]
[0.98125349 0.98123488]
[0.98125584 0.98123541]
[0.98125677 0.98123485]
[0.98125765 0.98123365]
[0.98125809 0.98123236]
[0.98125955 0.98123079]
[0.98126171 0.98122897]
[0.98126243 0.98122813]
[0.9812628  0.98122748]
[0.98126274 0.98122815]
[0.98126276 0.98122899]
[0.98126317 0.98122923]
[0.98126409 0.98122909]
[0.98126483 0.9812294 ]
[0.98126644 0.98122884]
[0.98126687 0.98122751]
[0.98126726 0.98122628]
[0.98126775 0.98122567]
[0.98126811 0.98122554]
[0.9812685 0.9812255]
[0.98126787 0.98122505]
[0.98126754 0.98122512]
[0.9812664  0.98122509]
[0.98126533 0.981224

[0.98124492 0.98129856]
[0.98124656 0.98130013]
[0.98124882 0.98130153]
[0.98125087 0.98130207]
[0.98125361 0.98130142]
[0.98125662 0.98130113]
[0.98125923 0.98130113]
[0.98126182 0.98130034]
[0.98126403 0.98129933]
[0.98126512 0.98129789]
[0.98126516 0.98129551]
[0.98126606 0.98129254]
[0.98126567 0.98128985]
[0.98126597 0.98128779]
[0.98126717 0.98128533]
[0.98126789 0.98128332]
[0.98126758 0.98128223]
[0.9812665 0.9812823]
[0.98126553 0.98128168]
[0.98126433 0.98128157]
[0.98126428 0.98128096]
[0.98126378 0.9812813 ]
[0.98126281 0.98128044]
[0.98126271 0.98127913]
[0.98126156 0.98127837]
[0.98126097 0.98127685]
[0.98125945 0.98127554]
[0.98125795 0.98127546]
[0.9812578  0.98127564]
[0.98125719 0.98127606]
[0.9812556  0.98127563]
[0.98125329 0.98127495]
[0.98125031 0.98127402]
[0.98124825 0.98127284]
[0.98124712 0.98127086]
[0.98124654 0.98126941]
[0.98124498 0.98126768]
[0.98124279 0.98126506]
[0.98124207 0.98126193]
[0.98124249 0.98125901]
[0.98124297 0.98125651]
[0.98124269 0.9812

[0.98124448 0.9812487 ]
[0.98124503 0.98124756]
[0.98124448 0.98124776]
[0.9812448  0.98124854]
[0.98124432 0.98124886]
[0.98124497 0.98124829]
[0.98124673 0.98124689]
[0.98124742 0.98124589]
[0.98124702 0.98124529]
[0.98124706 0.98124508]
[0.98124619 0.98124516]
[0.98124531 0.98124633]
[0.98124505 0.98124703]
[0.98124489 0.9812487 ]
[0.98124585 0.98125068]
[0.98124632 0.9812529 ]
[0.98124729 0.98125472]
[0.98124841 0.98125691]
[0.98124961 0.98125788]
[0.98125178 0.98125926]
[0.98125259 0.98126055]
[0.98125444 0.98126148]
[0.98125563 0.98126116]
[0.98125701 0.98126108]
[0.98125868 0.98126094]
[0.98126101 0.98125974]
[0.98126239 0.98125893]
[0.98126406 0.98125936]
[0.98126503 0.98126021]
[0.98126531 0.98126048]
[0.98126611 0.98126085]
[0.98126669 0.98126043]
[0.98126666 0.98125987]
[0.98126744 0.98125871]
[0.98126742 0.98125825]
[0.98126749 0.98125838]
[0.98126828 0.98125803]
[0.98126784 0.9812566 ]
[0.98126688 0.98125651]
[0.98126515 0.98125692]
[0.9812644  0.98125619]
[0.98126443 0.98

[0.98121765 0.98128681]
[0.9812196  0.98128767]
[0.98122067 0.98128841]
[0.98122154 0.9812896 ]
[0.98122189 0.98129114]
[0.98122206 0.98129341]
[0.98122129 0.98129489]
[0.98122147 0.98129621]
[0.98122224 0.98129749]
[0.98122302 0.98129796]
[0.98122321 0.98129892]
[0.98122354 0.98130002]
[0.98122502 0.98130026]
[0.98122558 0.98130097]
[0.98122664 0.98130105]
[0.98122808 0.98130076]
[0.98123016 0.98130098]
[0.98123241 0.98130078]
[0.98123452 0.98130051]
[0.98123557 0.98129983]
[0.9812375  0.98129877]
[0.98123938 0.98129835]
[0.98124032 0.98129862]
[0.98124148 0.98129871]
[0.98124311 0.98129897]
[0.98124432 0.98129923]
[0.98124479 0.98130011]
[0.98124546 0.98129953]
[0.98124605 0.98129858]
[0.98124735 0.98129732]
[0.98124845 0.98129588]
[0.98124904 0.98129346]
[0.98125044 0.98129044]
[0.98125147 0.98128695]
[0.98125215 0.98128491]
[0.98125245 0.98128196]
[0.98125276 0.98127945]
[0.98125357 0.98127609]
[0.98125386 0.98127332]
[0.98125491 0.98127042]
[0.98125514 0.98126769]
[0.98125521 0.98

...

[0.98129026 0.98126804]
[0.98128965 0.98126719]
[0.98129009 0.98126571]
[0.98129063 0.98126445]
[0.98129167 0.98126426]
[0.98129235 0.98126467]
[0.98129331 0.98126392]
[0.98129398 0.98126287]
[0.9812955  0.98126234]
[0.98129655 0.98126107]
[0.98129676 0.98125905]
[0.98129718 0.98125651]
[0.98129641 0.98125499]
[0.98129627 0.9812534 ]
[0.98129519 0.98125126]
[0.981293   0.98124839]
[0.98129013 0.98124688]
[0.98128656 0.98124526]
[0.98128288 0.98124463]
[0.98127949 0.98124443]
[0.98127762 0.98124337]
[0.98127582 0.98124174]
[0.9812739  0.98123942]
[0.98127154 0.98123685]
[0.98126861 0.98123522]
[0.98126619 0.98123385]
[0.98126349 0.98123201]
[0.98126195 0.98123055]
[0.98126094 0.98122867]
[0.98126084 0.98122642]
[0.98126014 0.98122576]
[0.98125866 0.9812262 ]
[0.98125795 0.98122701]
[0.98125823 0.9812288 ]
[0.98125801 0.98123129]
[0.98125717 0.98123409]
[0.98125591 0.9812367 ]
[0.98125505 0.98123855]
[0.98125372 0.9812397 ]
[0.98125374 0.98124162]
[0.98125344 0.98124346]
[0.98125244 0.98

[0.98125203 0.98125862]
[0.9812531 0.9812552]
[0.98125336 0.98125275]
[0.98125466 0.98125074]
[0.98125626 0.98124957]
[0.98125669 0.9812487 ]
[0.98125771 0.9812479 ]
[0.9812593  0.98124656]
[0.98125953 0.98124439]
[0.98126022 0.98124307]
[0.98125982 0.98124188]
[0.98125862 0.98124124]
[0.98125705 0.98124141]
[0.98125467 0.98124089]
[0.98125365 0.98123956]
[0.98125233 0.98123823]
[0.98125127 0.98123616]
[0.98125051 0.98123506]
[0.98124959 0.98123335]
[0.98124981 0.98123171]
[0.98124999 0.98123049]
[0.98124996 0.98122851]
[0.98124912 0.98122705]
[0.98124768 0.98122677]
[0.98124635 0.98122594]
[0.98124487 0.9812255 ]
[0.98124308 0.9812254 ]
[0.9812427 0.9812258]
[0.98124267 0.98122657]
[0.98124239 0.98122698]
[0.98124275 0.98122697]
[0.98124242 0.98122779]
[0.98124214 0.98122872]
[0.98124194 0.98123063]
[0.98124211 0.98123195]
[0.98124231 0.98123355]
[0.98124347 0.98123431]
[0.98124369 0.9812361 ]
[0.98124511 0.98123699]
[0.98124753 0.98123701]
[0.98125077 0.98123805]
[0.98125268 0.981238

[0.98127143 0.98130101]
[0.98127    0.98129999]
[0.98126987 0.98129853]
[0.98126913 0.98129816]
[0.9812679  0.98129781]
[0.98126678 0.98129709]
[0.98126681 0.98129565]
[0.98126735 0.98129402]
[0.98126808 0.98129353]
[0.98126814 0.9812927 ]
[0.9812673  0.98129071]
[0.98126619 0.98128933]
[0.98126593 0.98128853]
[0.98126477 0.98128727]
[0.98126364 0.98128582]
[0.98126283 0.98128339]
[0.98126308 0.98128012]
[0.98126433 0.98127726]
[0.98126599 0.98127541]
[0.98126728 0.98127399]
[0.98126753 0.98127248]
[0.98126706 0.98127029]
[0.98126673 0.98126751]
[0.98126614 0.98126615]
[0.98126483 0.9812645 ]
[0.98126332 0.98126344]
[0.98126123 0.98126258]
[0.98125949 0.98126203]
[0.98125736 0.98126222]
[0.98125522 0.98126207]
[0.98125375 0.98126139]
[0.98125282 0.98126028]
[0.98125277 0.98126016]
[0.98125361 0.98125921]
[0.98125323 0.98125753]
[0.98125317 0.98125717]
[0.98125424 0.98125627]
[0.98125517 0.98125477]
[0.98125555 0.98125321]
[0.98125641 0.98125122]
[0.98125696 0.98125058]
[0.98125844 0.98

[0.98127032 0.98123888]
[0.98126972 0.98123746]
[0.98126997 0.9812354 ]
[0.98126921 0.98123399]
[0.98126889 0.98123359]
[0.98126794 0.98123386]
[0.98126759 0.98123407]
[0.98126707 0.98123466]
[0.98126589 0.98123466]
[0.98126419 0.98123466]
[0.98126236 0.9812351 ]
[0.98126188 0.98123606]
[0.98126125 0.98123702]
[0.98126171 0.9812382 ]
[0.9812611  0.98123936]
[0.98126118 0.98124156]
[0.9812613  0.98124334]
[0.98126055 0.98124521]
[0.98126024 0.98124716]
[0.98125931 0.98124788]
[0.98125842 0.98124814]
[0.98125797 0.98124734]
[0.98125851 0.98124644]
[0.98125834 0.98124551]
[0.9812592  0.98124521]
[0.98125943 0.98124603]
[0.98126031 0.98124679]
[0.98126026 0.9812483 ]
[0.98126078 0.98124941]
[0.98126144 0.98125046]
[0.98126174 0.98125113]
[0.98126109 0.98125078]
[0.9812609  0.98125145]
[0.98126168 0.98125172]
[0.98126308 0.98125221]
[0.98126488 0.98125301]
[0.9812672  0.98125403]
[0.98126865 0.98125469]
[0.98126889 0.98125554]
[0.98126912 0.98125707]
[0.98126993 0.98125743]
[0.98126968 0.98

[0.981276   0.98125558]
[0.98127738 0.98125727]
[0.98127874 0.98125905]
[0.98128068 0.98126164]
[0.98128161 0.981264  ]
[0.98128223 0.98126627]
[0.98128247 0.98126806]
[0.98128317 0.98127004]
[0.98128323 0.98127119]
[0.98128242 0.98127271]
[0.98128107 0.98127436]
[0.98127874 0.98127649]
[0.98127669 0.98127745]
[0.98127472 0.98127887]
[0.98127298 0.9812805 ]
[0.98127148 0.98128266]
[0.98127086 0.98128512]
[0.98126925 0.98128655]
[0.98126682 0.98128868]
[0.98126506 0.98129105]
[0.9812624  0.98129329]
[0.98126088 0.98129398]
[0.98125871 0.98129355]
[0.98125612 0.98129218]
[0.98125281 0.98129088]
[0.98125013 0.98128973]
[0.98124831 0.98128882]
[0.98124746 0.98128684]
[0.98124572 0.98128441]
[0.98124522 0.98128123]
[0.98124417 0.98127822]
[0.98124352 0.98127549]
[0.98124291 0.98127295]
[0.98124326 0.98127173]
[0.9812439  0.98127044]
[0.98124388 0.98126936]
[0.98124421 0.98126747]
[0.98124526 0.9812658 ]
[0.98124601 0.98126494]
[0.98124684 0.98126318]
[0.98124654 0.98126253]
[0.98124622 0.98

In [10]:
for i in range(sim_steps):
    time+=dt
    timestamps.append(time)
    current_noise = noisy_channel(current_noise)
    hidden_states = build_hidden_states(couplings=couplings_matrix,current_states=population_states,inputs=inputs,
                                    ionic_noises=current_noise,external_sources = I0)
    population_states = update_states(hidden_states = hidden_states,current_states=population_states)